In [30]:
import numpy as np
import pandas as pd
from scipy.stats import norm


def black_scholes_call(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    call_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    return call_price


def calculate_call_prices(prices, strikes, days_to_expire, r=0.05, sigma=None):
    prices = np.array(prices)
    n_days = len(prices)
    if sigma is None:
        sigma = np.std(daily_returns) * np.sqrt(252)  # Annualized volatility

    results = []
    for strike in strikes:
        for days in days_to_expire:
            call_prices = []
            for i in range(n_days):
                S = prices[i]
                T = (days - i) / 252  # Update time to expiration
                call_price = black_scholes_call(S, strike, T, r, sigma)
                call_prices.append(call_price)

            call_prices = np.array(call_prices)
            daily_pct_changes = np.diff(call_prices) / call_prices[:-1]
            daily_pct_changes = np.insert(
                daily_pct_changes, 0, np.nan
            )  # Insert 'NA' for day 0

            results.extend(
                zip(
                    np.repeat(days, n_days),
                    np.repeat(strike, n_days),
                    range(n_days),
                    call_prices,
                    daily_pct_changes,
                )
            )

    df = pd.DataFrame(
        results,
        columns=["Expire", "Strike", "Day", "Call Price", "Call Price Pct Change"],
    )

    return df


def calculate_daily_returns(prices):
    prices = np.array(prices)
    n_days = len(prices)

    daily_returns = np.diff(prices) / prices[:-1]
    return daily_returns


# Example usage
r = 0.01  # 無風險利率（1%）
# sigma = 0.2  # 波動率（20%）
sigma = None
prices = [900, 850, 800, 820, 850, 880, 900, 920, 940, 950]  # Stock prices for 10 days
strikes = [850, 900, 950]  # Multiple strike prices
days_to_expire = [30, 60, 90]  # Multiple expiration dates (in days)


# daily_returns = calculate_daily_returns(prices)
# pd.DataFrame(daily_returns).pivot_table()
# df = pd.DataFrame(
#     {
#         "returns": daily_returns,
#         "price": prices[1:],
#         "day": range(1, len(prices)),
#     }
# )
# # print()
# df.pivot_table(columns="day", values=["returns", "price"])

df = calculate_call_prices(prices, strikes, days_to_expire)
df_pivot = df.pivot_table(
    index=["Expire", "Strike"], columns="Day", values="Call Price Pct Change"
)
# df_pivot.loc[("NA (Stock)", "NA (Stock)"), 1:] = daily_returns
# df_pivot.columns.name = "Day"

# pd.options.display.float_format = "{:.2f}".format
# df_pivot

Day               1     2    3    4    5    6    7    8    9
Expire Strike                                               
30     850    -0.32 -0.39 0.19 0.29 0.26 0.15 0.14 0.13 0.06
       900    -0.36 -0.43 0.21 0.33 0.30 0.17 0.16 0.15 0.06
       950    -0.40 -0.47 0.23 0.37 0.33 0.18 0.18 0.17 0.06
60     850    -0.25 -0.28 0.13 0.20 0.18 0.11 0.10 0.10 0.04
       900    -0.27 -0.31 0.14 0.22 0.20 0.12 0.11 0.11 0.05
       950    -0.29 -0.33 0.16 0.24 0.22 0.13 0.12 0.12 0.05
90     850    -0.21 -0.24 0.11 0.16 0.15 0.09 0.09 0.08 0.04
       900    -0.23 -0.25 0.12 0.17 0.16 0.10 0.09 0.09 0.04
       950    -0.24 -0.27 0.12 0.19 0.17 0.10 0.10 0.09 0.04

In [37]:
import numpy as np
import pandas as pd
from scipy.stats import norm
import plotly.express as px

# ... (previous code remains the same)

# Example usage
r = 0.01  # 無風險利率（1%）
# sigma = 0.2  # 波動率（20%）
sigma = None
prices = [900, 850, 800, 820, 850, 880, 900, 920, 940, 950]  # Stock prices for 10 days
strikes = [850, 900, 950]  # Multiple strike prices
days_to_expire = [30, 60, 90]  # Multiple expiration dates (in days)

daily_returns = calculate_daily_returns(prices)

df = calculate_call_prices(prices, strikes, days_to_expire, r, sigma)

# Create a new DataFrame with stock price percentage changes
stock_df = pd.DataFrame(
    {
        "Day": range(1, len(prices)),
        "Stock Price Pct Change": daily_returns,
        "Expire": 0,  # Set a default value for Expire column
        "Strike": 0,  # Set a default value for Strike column
    }
)

# Combine the stock and call price DataFrames
combined_df = pd.concat([df, stock_df], axis=0)

# print(combined_df)

# Create the scatter chart using Plotly Express
fig = px.scatter(
    combined_df,
    x="Day",
    y=["Call Price Pct Change", "Stock Price Pct Change"],
    color="Expire",
    symbol="Strike",
    hover_data={"Call Price": ":.2f", "Strike": True, "Expire": True},
    labels={"Day": "Day", "value": "Pct Change", "Expire": "DTE", "Strike": "Strike"},
    title="Stock Price and Call Price Percentage Changes",
    height=600,
)

fig.show()

In [46]:
import plotly.graph_objects as go

# ... (previous code remains the same)

# Create a DataFrame for stock prices
stock_prices_df = pd.DataFrame({"Day": range(len(prices)), "Stock Price": prices})

# Create line charts using Plotly Express
fig_stock = px.line(
    stock_prices_df,
    x="Day",
    y="Stock Price",
    title="Stock Price vs Time",
)
# fig_stock = go.Scatter(
#     x=stock_prices_df['Day'],
#     y=stock_prices_df['Stock Price'],
#     mode='lines',
#     name='Stock Price',
#     line=dict(color='black', width=3, dash='solid')
# )

fig_calls = px.line(
    df,
    x="Day",
    y="Call Price",
    color="Expire",
    line_dash="Strike",
    hover_data={"Call Price": ":.2f", "Strike": True, "Expire": True},
    labels={"Day": "Day", "Call Price": "Call Price"},
    title="Call Prices vs Time",
)

# Create a subplot with the stock price and call prices
fig = go.Figure(data=fig_stock.data + fig_calls.data)
fig.update_layout(
    xaxis=dict(title="Day"),
    yaxis=dict(title="Stock Price"),
    yaxis2=dict(title="Call Price", overlaying="y", side="right"),
    legend=dict(
        title="", orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1
    ),
    height=600,
)

# Assign the call price traces to the secondary y-axis
for trace in fig.data[1:]:
    trace.yaxis = "y2"

fig.show()